<a href="https://colab.research.google.com/github/maikelele/traffic-sign-recognition/blob/dataset/yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def conv_block(x, filters, kernel_size, strides=1, activation=True):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(x, filters):
    shortcut = x
    x = conv_block(x, filters // 2, 1)
    x = conv_block(x, filters, 3)
    x = layers.Add()([shortcut, x])
    return x

def csp_block(x, filters, num_blocks):
    x = conv_block(x, filters, 3, strides=2)
    route = x
    route = conv_block(route, filters // 2, 1)

    x = conv_block(x, filters // 2, 1)
    for _ in range(num_blocks):
        x = residual_block(x, filters // 2)

    x = conv_block(x, filters // 2, 1)
    x = layers.Concatenate()([x, route])
    x = conv_block(x, filters, 1)
    return x

def create_panet(features):
    f3, f4, f5 = features
    p5 = conv_block(f5, 256, 1)
    p5 = layers.UpSampling2D(2)(p5)
    p4 = layers.Concatenate()([p5, f4])
    p4 = conv_block(p4, 256, 1)
    p4 = layers.UpSampling2D(2)(p4)
    p3 = layers.Concatenate()([p4, f3])
    p3 = conv_block(p3, 256, 1)
    return p3, p4, p5

def yolo_head(x, num_classes):
    x = conv_block(x, 512, 3)
    x = conv_block(x, 3 * (num_classes + 5), 1, activation=False)
    return x

def create_yolov5(input_shape, num_classes):
    inputs = layers.Input(input_shape)
    x = conv_block(inputs, 32, 3)

    x = csp_block(x, 64, 1)
    x = csp_block(x, 128, 2)
    x = csp_block(x, 256, 8)
    f3 = x
    x = csp_block(x, 512, 8)
    f4 = x
    x = csp_block(x, 1024, 4)
    f5 = x

    features = create_panet([f3, f4, f5])

    yolo_outputs = [yolo_head(feature, num_classes) for feature in features]

    model = models.Model(inputs, yolo_outputs)
    return model

input_shape = (640, 640, 3)
num_classes = 264
model = create_yolov5(input_shape, num_classes)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 640, 640, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 640, 640, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 640, 640, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 640, 640, 32)         0         ['batch_normalization[0][0

In [2]:
import glob

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="J1WZ9Alz86gSqL19G0sk")
project = rf.workspace("google-8fa6i").project("trafficsignrecognition-s1x6u")
version = project.version(2)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...


In [17]:
def load_image_and_labels(image_path, label_path, max_labels=50):
    # Read and process the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [640, 640])
    image = image / 255.0  # Normalize to [0, 1]

    # Read the label file and convert it into a tensor of shape [num_labels, 5]
    labels = tf.io.read_file(label_path)
    labels = tf.strings.split(labels, sep='\n')
    labels = tf.strings.split(labels, sep=' ')
    labels = tf.ragged.constant([tf.strings.to_number(label, tf.float32) for label in labels if tf.strings.length(label) > 0])
    labels = labels.to_tensor(shape=[max_labels, 5])

    return image, labels


def prepare_dataset(image_paths, label_paths, batch_size, max_labels=50):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
    dataset = dataset.map(lambda x, y: load_image_and_labels(x, y, max_labels), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


def get_file_paths(directory, pattern="*.jpg"):
    """Retrieve file paths based on the given pattern."""
    return glob.glob(f"{directory}/{pattern}")

# Define directories
image_directory = "/content/Trafficsignrecognition-2/train/images"
label_directory = "/content/Trafficsignrecognition-2/train/labels"

# Get lists of image and label paths
train_image_paths = get_file_paths(image_directory, pattern="*.jpg")
train_label_paths = get_file_paths(label_directory, pattern="*.txt")
train_image_paths.sort()
train_label_paths.sort()

batch_size = 4
train_dataset = prepare_dataset(train_image_paths, train_label_paths, batch_size)

OperatorNotAllowedInGraphError: in user code:

    File "<ipython-input-17-da79428a88dd>", line 20, in None  *
        lambda x, y: load_image_and_labels(x, y, max_labels)
    File "<ipython-input-17-da79428a88dd>", line 12, in load_image_and_labels  *
        labels = tf.ragged.constant([tf.strings.to_number(label, tf.float32) for label in labels if tf.strings.length(label) > 0])

    OperatorNotAllowedInGraphError: Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more i

In [6]:
def yolo_loss(y_true, y_pred):
    # Implement the loss function here
    # This is just a placeholder function
    return tf.reduce_sum((y_true - y_pred) ** 2)


In [7]:
optimizer = tf.optimizers.Adam(learning_rate=0.001)
epochs = 10

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = model(x_batch_train, training=True)  # Logits for this minibatch
            # Compute the loss value for this minibatch
            loss_value = yolo_loss(y_batch_train, reconstructed)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 100 batches.
        if step % 100 == 0:
            print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss_value)))
            print("Seen so far: %d samples" % ((step + 1) * batch_size))



Start of epoch 0


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cannot batch tensors with different shapes in component 1. First element had shape [20] and element 1 had shape [15]. [Op:IteratorGetNext] name: 